In [3]:
# 得到小于2s的项

import pandas as pd

# File paths
llm_file_path = '/data1/dxw_data/llm/tiktok/accuracy/DatafromLLM.xlsx'
ra_file_path = '/data1/dxw_data/llm/tiktok/accuracy/DatafromRA.xlsx'

# Load DatafromLLM.xlsx
llm_df = pd.read_excel(llm_file_path)

# Filter rows where 'Time' is less than or equal to 2
filtered_llm_df = llm_df[llm_df['Time'] <= 2][['UserID', 'Video']]

# Save the filtered data to a new Excel file
filtered_llm_df.to_csv('/data1/dxw_data/llm/tiktok/accuracy/FilteredDatafromLLM.csv', index=False)

# Load DatafromRA.xlsx
ra_df = pd.read_excel(ra_file_path)

# Filter rows where '是否有效曝光' (Effective Exposure) is '否' (No)
filtered_ra_df = ra_df[ra_df['是否有效曝光'] == '否'][['UserID', 'Video']]

# Save the filtered data to a new Excel file
filtered_ra_df.to_csv('/data1/dxw_data/llm/tiktok/accuracy/FilteredDatafromRA.csv', index=False)

print("Filtered data has been saved successfully.")


Filtered data has been saved successfully.


In [2]:
# 得到大于2s的项

import pandas as pd

# Read DatafromRA.xlsx
data_ra = pd.read_excel('/data1/dxw_data/llm/tiktok/accuracy/DatafromRA.xlsx')

# Filter rows where "是否有效曝光" is "是"
data_ra_valid = data_ra[data_ra['是否有效曝光'] == '是']

# Group by UserID and Video, then create lists for Main Category and Sub Category
grouped_ra = data_ra_valid.groupby(['UserID', 'Video'])
ra_result = grouped_ra.agg({
    'Main Category': lambda x: list(x),
    'Sub Category': lambda x: list(x)
}).reset_index()

# Rename columns for clarity
ra_result.columns = ['UserID', 'Video', 'Main_Category_list', 'Sub_Category_list']

# Save the result to a new CSV
ra_result.to_csv('/data1/dxw_data/llm/tiktok/accuracy/DatafromRA_Main_Categories.csv', index=False)

# Read DatafromLLM.xlsx
data_llm = pd.read_excel('/data1/dxw_data/llm/tiktok/accuracy/DatafromLLM.xlsx')

# Filter rows where Time is greater than 2
data_llm_valid = data_llm[data_llm['Time'] > 2]

# Group by UserID and Video, then create a list for Topic
grouped_llm = data_llm_valid.groupby(['UserID', 'Video'])
llm_result = grouped_llm.agg({
    'Topic': lambda x: list(x)
}).reset_index()

# Rename column for clarity
llm_result.columns = ['UserID', 'Video', 'Main_Category_list']

# Save the result to a new CSV
llm_result.to_csv('/data1/dxw_data/llm/tiktok/accuracy/DatafromLLM_Main_Categories.csv', index=False)

print("Data processing completed and CSV files saved.")


Data processing completed and CSV files saved.


In [1]:
# 计算准确度 小于2s

import pandas as pd

# File paths
llm_file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/FilteredDatafromLLM.csv'
ra_file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/FilteredDatafromRA.csv'
output_file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/MatchedData_2.csv'

# Read the CSV files
llm_df = pd.read_csv(llm_file_path)
ra_df = pd.read_csv(ra_file_path)

# Count occurrences of UserID, Video pairs in each file
llm_counts = llm_df.groupby(['UserID', 'Video']).size().reset_index(name='LLM_num')
ra_counts = ra_df.groupby(['UserID', 'Video']).size().reset_index(name='RA_num')

# Merge the counts on UserID and Video, filling NaN with 0
merged_df = pd.merge(llm_counts, ra_counts, on=['UserID', 'Video'], how='outer').fillna(0)

# Convert the counts to integers (as they were initially counts)
merged_df['LLM_num'] = merged_df['LLM_num'].astype(int)
merged_df['RA_num'] = merged_df['RA_num'].astype(int)

# Save the result to a new CSV file
merged_df.to_csv(output_file_path, index=False)

print(f"Matched data has been saved to {output_file_path}")


Matched data has been saved to /data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/MatchedData_2.csv


In [1]:
# 大类匹配

import pandas as pd
import ast
import re

# Define the category groups
categories = [
    ["Product Guide", "Product Review", "Outfit Recommendations"],
    ["Educational/Informational", "Popular Science", "Medical Health", "Education and Campus", "Workplace/Interpersonal Relationships", "Technology"],
    ["Plot Type", "Movie Commentary", "Film/Variety Editing", "Short Skits", "Animation/Anime"],
    ["Leisure and Comedy", "Creative Editing/Dubbing", "Satirical Parody", "Art Creation", "Street Interviews", "Landscape Photography", "Cute Pets"],
    ["Life Sharing", "Food", "Vlog/Insights Sharing", "Travel", "Fitness and Beauty", "Skill Sharing", "Home Life"],
    ["Appearance", "Dance", "Music"],
    ["Advertising", "Local Culture and Tourism", "Public Service Advertising"],
    ["Celebrity Entertainment"],
    ["News and Current Events", "Social and Political News"],
    ["Games"],
    ["Sports"],
    ["Automobiles"],
    ["Finance"],
    ["Others"]
]

# Create a dictionary to map each category to its group index
category_to_group = {category: index for index, group in enumerate(categories) for category in group}

# Read the CSV files
llm_df = pd.read_csv('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromLLM_Main_Categories.csv')
ra_df = pd.read_csv('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromRA_Main_Categories.csv')

# Function to map categories to their group indices
def map_categories_to_groups(category_list, category_to_group):
    return [category_to_group.get(cat, -1) for cat in category_list]

# Preprocess the string to ensure it's a valid Python literal
def preprocess_category_string(category_string):
    # Remove newlines and extra spaces
    cleaned_string = re.sub(r'\s+', ' ', category_string.strip())
    # Ensure it's a list format
    if not (cleaned_string.startswith("[") and cleaned_string.endswith("]")):
        cleaned_string = f"[{cleaned_string}]"
    return cleaned_string

# Function to safely evaluate category strings
def safe_eval(category_string):
    try:
        return ast.literal_eval(category_string)
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing string: {category_string}")
        return []

# Function to calculate sequence match with the shorter list matching against the longer list
def calculate_sequence_match(llm_list, ra_list, category_to_group):
    llm_groups = map_categories_to_groups(llm_list, category_to_group)
    ra_groups = map_categories_to_groups(ra_list, category_to_group)

    # Determine which list is shorter
    if len(ra_groups) <= len(llm_groups):
        short_groups, long_groups = ra_groups, llm_groups
    else:
        short_groups, long_groups = llm_groups, ra_groups

    # Sequence matching using the shorter list
    sequence_num = 0
    long_index = 0

    for short_group in short_groups:
        while long_index < len(long_groups):
            if long_groups[long_index] == short_group:
                sequence_num += 1
                long_index += 1
                break
            long_index += 1

    return sequence_num

# Prepare a list to store the results
results = []

# Iterate over the rows in ra_df
for _, ra_row in ra_df.iterrows():
    user_id = ra_row['UserID']
    video_id = ra_row['Video']

    # Find the corresponding LLM row
    llm_row = llm_df[(llm_df['UserID'] == user_id) & (llm_df['Video'] == video_id)]

    if not llm_row.empty:
        llm_categories = llm_row.iloc[0]['Main_Category_list']
        ra_categories = ra_row['Main_Category_list']

        # Preprocess and evaluate category strings
        llm_list = safe_eval(preprocess_category_string(llm_categories))
        ra_list = safe_eval(preprocess_category_string(ra_categories))

        # Calculate all_num and sequence_num
        all_num = min(len(ra_list), len(llm_list))  # Use the length of the shorter list
        sequence_num = calculate_sequence_match(llm_list, ra_list, category_to_group)

        # Append the results
        results.append((user_id, video_id, all_num, sequence_num))

# Convert the results to a DataFrame
results_df = pd.DataFrame(results, columns=['UserID', 'Video', 'all_num', 'sequence_num'])

# Save the results to a new CSV file
output_file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/MatchedData_CategorySequences_single.csv'
results_df.to_csv(output_file_path, index=False)

print(f"Results saved to {output_file_path}")


Error parsing string: ['Others', 'Life Sharing', 'Life Sharing', 'Life Sharing', nan, 'Life Sharing', 'Leisure and Comedy', 'Leisure and Comedy', 'Leisure and Comedy', 'Plot Type', 'Leisure and Comedy', nan, 'Leisure and Comedy', 'Leisure and Comedy', 'Leisure and Comedy', 'Leisure and Comedy', 'Leisure and Comedy', 'Life Sharing', 'Others', 'Leisure and Comedy']
Error parsing string: ['Leisure and Comedy', 'News and Current Events', 'News and Current Events', 'Leisure and Comedy', 'Leisure and Comedy', 'Others', nan, nan, 'Life Sharing', 'Plot Type', nan, 'Leisure and Comedy', 'Life Sharing', 'Appearance', 'Life Sharing', nan, 'Life Sharing', 'Sports', 'Life Sharing', 'News and Current Events', nan, 'Sports', 'Sports', 'Sports', 'Sports']
Error parsing string: ['Life Sharing', 'Appearance', 'Leisure and Comedy', 'Appearance', 'Appearance', 'Leisure and Comedy', nan, 'Appearance', 'Appearance']
Error parsing string: ['Life Sharing', 'Leisure and Comedy', 'Others', 'Leisure and Comedy',

In [4]:
# 子类匹配

import pandas as pd
import ast
import re

# Read the CSV files
llm_df = pd.read_csv('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromLLM_Main_Categories.csv')
ra_df = pd.read_csv('/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/DatafromRA_Main_Categories.csv')

# Preprocess the string to ensure it's a valid Python literal
def preprocess_category_string(category_string):
    # Remove newlines and extra spaces
    cleaned_string = re.sub(r'\s+', ' ', category_string.strip())
    # Ensure it's a list format
    if not (cleaned_string.startswith("[") and cleaned_string.endswith("]")):
        cleaned_string = f"[{cleaned_string}]"
    return cleaned_string

# Function to safely evaluate category strings
def safe_eval(category_string):
    try:
        return ast.literal_eval(category_string)
    except (ValueError, SyntaxError) as e:
        print(f"Error parsing string: {category_string}")
        return []

# Function to calculate sequence match with the shorter list matching against the longer list
def calculate_sequence_match(llm_list, ra_list):
    # Determine which list is shorter
    if len(ra_list) <= len(llm_list):
        short_list, long_list = ra_list, llm_list
    else:
        short_list, long_list = llm_list, ra_list

    # Sequence matching using the shorter list
    sequence_num = 0
    long_index = 0

    for short_item in short_list:
        while long_index < len(long_list):
            if long_list[long_index] == short_item:
                sequence_num += 1
                long_index += 1
                break
            long_index += 1

    return sequence_num

# Prepare a list to store the results
results = []

# Iterate over the rows in ra_df
for _, ra_row in ra_df.iterrows():
    user_id = ra_row['UserID']
    video_id = ra_row['Video']

    # Find the corresponding LLM row
    llm_row = llm_df[(llm_df['UserID'] == user_id) & (llm_df['Video'] == video_id)]

    if not llm_row.empty:
        llm_categories = llm_row.iloc[0]['Main_Category_list']
        ra_categories = ra_row['Main_Category_list']

        # Preprocess and evaluate category strings
        llm_list = safe_eval(preprocess_category_string(llm_categories))
        ra_list = safe_eval(preprocess_category_string(ra_categories))

        # Calculate all_num and sequence_num
        all_num = min(len(ra_list), len(llm_list))  # Use the length of the shorter list
        sequence_num = calculate_sequence_match(llm_list, ra_list)

        # Append the results
        results.append((user_id, video_id, all_num, sequence_num))

# Convert the results to a DataFrame
results_df = pd.DataFrame(results, columns=['UserID', 'Video', 'all_num', 'sequence_num'])

# Save the results to a new CSV file
output_file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/MatchedData_CategorySequences_single2.csv'
results_df.to_csv(output_file_path, index=False)

print(f"Results saved to {output_file_path}")


Error parsing string: ['Others', 'Life Sharing', 'Life Sharing', 'Life Sharing', nan, 'Life Sharing', 'Leisure and Comedy', 'Leisure and Comedy', 'Leisure and Comedy', 'Plot Type', 'Leisure and Comedy', nan, 'Leisure and Comedy', 'Leisure and Comedy', 'Leisure and Comedy', 'Leisure and Comedy', 'Leisure and Comedy', 'Life Sharing', 'Others', 'Leisure and Comedy']
Error parsing string: ['Leisure and Comedy', 'News and Current Events', 'News and Current Events', 'Leisure and Comedy', 'Leisure and Comedy', 'Others', nan, nan, 'Life Sharing', 'Plot Type', nan, 'Leisure and Comedy', 'Life Sharing', 'Appearance', 'Life Sharing', nan, 'Life Sharing', 'Sports', 'Life Sharing', 'News and Current Events', nan, 'Sports', 'Sports', 'Sports', 'Sports']
Error parsing string: ['Life Sharing', 'Appearance', 'Leisure and Comedy', 'Appearance', 'Appearance', 'Leisure and Comedy', nan, 'Appearance', 'Appearance']
Error parsing string: ['Life Sharing', 'Leisure and Comedy', 'Others', 'Leisure and Comedy',

In [3]:
# 计算具体准确率

In [3]:
import pandas as pd

# Load the CSV file
file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/MatchedData_2.csv'
data = pd.read_csv(file_path)

# Calculate totals for each UserID and Video combination
user_video_totals = data.groupby(['UserID', 'Video']).agg({'LLM_num': 'sum', 'RA_num': 'sum'})

# Calculate accuracy with condition for each UserID and Video
user_video_totals['accuracy'] = user_video_totals.apply(
    lambda row: row['RA_num'] / row['LLM_num'] if row['LLM_num'] >= row['RA_num'] else row['LLM_num'] / row['RA_num'],
    axis=1
)

# Calculate totals and average match rate for each UserID
user_totals = data.groupby('UserID').agg({'LLM_num': 'sum', 'RA_num': 'sum'})

# Calculate accuracy for each UserID with condition
user_totals['accuracy'] = user_totals.apply(
    lambda row: row['RA_num'] / row['LLM_num'] if row['LLM_num'] >= row['RA_num'] else row['LLM_num'] / row['RA_num'],
    axis=1
)

# Calculate average match rate for each UserID
average_match_rate = user_video_totals.groupby('UserID')['accuracy'].mean()

# Write results to a text file
output_file_path = 'accuracy_results_2.txt'
with open(output_file_path, 'w') as f:
    f.write("Totals and Accuracy for Each UserID and Video Combination:\n")
    f.write(user_video_totals.to_string())
    f.write("\n\nTotals and Accuracy for Each UserID:\n")
    f.write(user_totals.to_string())
    f.write("\n\nAverage Match Rate for Each UserID:\n")
    f.write(average_match_rate.to_string())

print(f"Results have been saved to {output_file_path}")


Results have been saved to accuracy_results_2.txt


In [5]:
import pandas as pd

# Load the CSV file
file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/MatchedData_CategorySequences_single.csv'
data = pd.read_csv(file_path)

# Calculate totals for each UserID and Video combination
user_video_totals = data.groupby(['UserID', 'Video']).agg({'all_num': 'sum', 'sequence_num': 'sum'})

# Calculate accuracy with condition for each UserID and Video
user_video_totals['accuracy'] = user_video_totals.apply(
    lambda row: row['sequence_num'] / row['all_num'] if row['all_num'] >= row['sequence_num'] else row['all_num'] / row['sequence_num'],
    axis=1
)

# Calculate totals and average match rate for each UserID
user_totals = data.groupby('UserID').agg({'all_num': 'sum', 'sequence_num': 'sum'})

# Calculate accuracy for each UserID with condition
user_totals['accuracy'] = user_totals.apply(
    lambda row: row['sequence_num'] / row['all_num'] if row['all_num'] >= row['sequence_num'] else row['all_num'] / row['sequence_num'],
    axis=1
)

# Calculate average match rate for each UserID
average_match_rate = user_video_totals.groupby('UserID')['accuracy'].mean()

# Write results to a text file
output_file_path = 'accuracy_results_singlematch_maincategory.txt'
with open(output_file_path, 'w') as f:
    f.write("Totals and Accuracy for Each UserID and Video Combination:\n")
    f.write(user_video_totals.to_string())
    f.write("\n\nTotals and Accuracy for Each UserID:\n")
    f.write(user_totals.to_string())
    f.write("\n\nAverage Match Rate for Each UserID:\n")
    f.write(average_match_rate.to_string())

print(f"Results have been saved to {output_file_path}")


Results have been saved to accuracy_results_singlematch_maincategory.txt


/tmp/ipykernel_793563/2952089787.py:12: RuntimeWarning: invalid value encountered in scalar divide
  lambda row: row['sequence_num'] / row['all_num'] if row['all_num'] >= row['sequence_num'] else row['all_num'] / row['sequence_num'],


In [6]:
import pandas as pd

# Load the CSV file
file_path = '/data1/dxw_data/llm/MKT_data_mining/Multimodal/image2text/accuracy/MatchedData_CategorySequences_single2.csv'
data = pd.read_csv(file_path)

# Calculate totals for each UserID and Video combination
user_video_totals = data.groupby(['UserID', 'Video']).agg({'all_num': 'sum', 'sequence_num': 'sum'})

# Calculate accuracy with condition for each UserID and Video
user_video_totals['accuracy'] = user_video_totals.apply(
    lambda row: row['sequence_num'] / row['all_num'] if row['all_num'] >= row['sequence_num'] else row['all_num'] / row['sequence_num'],
    axis=1
)

# Calculate totals and average match rate for each UserID
user_totals = data.groupby('UserID').agg({'all_num': 'sum', 'sequence_num': 'sum'})

# Calculate accuracy for each UserID with condition
user_totals['accuracy'] = user_totals.apply(
    lambda row: row['sequence_num'] / row['all_num'] if row['all_num'] >= row['sequence_num'] else row['all_num'] / row['sequence_num'],
    axis=1
)

# Calculate average match rate for each UserID
average_match_rate = user_video_totals.groupby('UserID')['accuracy'].mean()

# Write results to a text file
output_file_path = 'accuracy_results_singlematch_subcategory.txt'
with open(output_file_path, 'w') as f:
    f.write("Totals and Accuracy for Each UserID and Video Combination:\n")
    f.write(user_video_totals.to_string())
    f.write("\n\nTotals and Accuracy for Each UserID:\n")
    f.write(user_totals.to_string())
    f.write("\n\nAverage Match Rate for Each UserID:\n")
    f.write(average_match_rate.to_string())

print(f"Results have been saved to {output_file_path}")


Results have been saved to accuracy_results_singlematch_subcategory.txt


/tmp/ipykernel_793563/669154580.py:12: RuntimeWarning: invalid value encountered in scalar divide
  lambda row: row['sequence_num'] / row['all_num'] if row['all_num'] >= row['sequence_num'] else row['all_num'] / row['sequence_num'],
